In [1]:
import numpy as np
import pandas as pd

In [2]:
def layer_size(X,Y, neurons):
    """
    X; Shape (m, X_w)
    Y; Shape (m, 1)
    """
    Input_layer = X.shape[1]    #Number of features in X
    Hidden_layer = neurons  # number of wanted neurons in hidden layer
    Output_Layer = Y.shape[1]  #Number features in Y
    
    return (Input_layer, Hidden_layer, Output_Layer)  #Return the defined size for each layer in the nn model

def init_parameters(Input_layer, Hidden_layer, Output_layer):
    """This function will initialize weights/bias parameters for the hidden & output layer.
    """
    
    #Weights are initialized as random values, to avoid symmetry. Bias are initialized as 0.
    W1 = np.random.randn(Input_layer, Hidden_layer) * 0.01
    b1 = np.zeros((1, Hidden_layer))
    W2 = np.random.randn(Hidden_layer, Output_layer) * 0.01
    b2 = np.zeros((1, Output_layer))
    
    
    #Store values in dictionary for easy access.
    parameters = {'W1':W1,
                  'b1':b1,
                  'W2':W2,
                  'b2':b2}
    
    return parameters


def forward_propagation(X, parameters):
    """
    Function to compute a prediction based on current values of weights & bias.
    """
    #We will use relu as activation function in the hidden layer.
    relu = lambda x: np.maximum(x,0) 
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    computation1 = np.dot(x_train, W1) + b1
    activation1 = relu(computation1)
    computation2 = np.dot(activation1, W2) + b2
    activation2 = relu(computation2)
    
    
    cache = {"computation1": computation1,
             "activation1": activation1,
             "computation2": computation2,
             "activation2": activation2}
    
    return activation2, cache
    
    
def cost_function(Y, activation2):
    """Function to compute cost.
    """
    
    m = Y.shape[0] # number of example

    #Mean squared error:
    MSE = (activation2 - Y)**2
    cost = (1/m)*np.sum(MSE)
    
    return cost

def backpropagation(parameters, cache, X, Y):
    """Compute gradients to optimize network.
    """
    m = X.shape[0]
    

    W2 = parameters["W2"]

    activation1 = cache["activation1"]
    activation2 = cache["activation2"]
    
    
     
    dZ2 = activation2 - y_train
    dW2 = np.dot(dZ2.T, activation1).T / m
    db2 = np.sum(dZ2, axis=0, keepdims=True)
    dZ1 = np.dot(dZ2, W2.T) * (1 - np.power(activation1, 2))
    dW1 = np.dot(dZ1.T, x_train).T / m
    db1 = np.sum(dZ1, axis=0, keepdims=True)
    

    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

def parameters_update(parameters, grads, learning_rate, optimizer, beta1, beta2, epsilon, t, L, v, s):
    """Update the current weights/bias with the chosen learning rate & using backprop function for gradients.
    """

    if optimizer != 'adam':
        
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]

        dW1 = grads["dW1"]
        db1 = grads["db1"]
        dW2 = grads["dW2"]
        db2 = grads["db2"]
        
        W1 = W1 - dW1 * learning_rate
        b1 = b1 - db1 * learning_rate
        W2 = W2 - dW2 * learning_rate
        b2 = b2 - db2 * learning_rate
        
    elif optimizer == 'adam':

        L = len(parameters) // 2                 # number of layers in the neural networks
        
        v_corrected = {}                         # Initializing first moment estimate, python dictionary
        s_corrected = {}                         # Initializing second moment estimate, python dictionary
        
        # Perform Adam update on all parameters
        for l in range(L):
            # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
            v["dW" + str(l+1)] = beta1*v["dW" + str(l+1)] + (1 - beta1)*grads['dW' + str(l+1)]
            v["db" + str(l+1)] = beta1*v["db" + str(l+1)] + (1 - beta1)*grads['db' + str(l+1)]

            # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
            v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)]/(1 - beta1**t)
            v_corrected["db" + str(l+1)] = v["db" + str(l+1)]/(1 - beta1**t)
            
            # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
            s["dW" + str(l+1)] = beta2*s["dW" + str(l+1)] + (1 - beta2)*np.square(grads['dW' + str(l+1)])
            s["db" + str(l+1)] = beta2*s["db" + str(l+1)] + (1 - beta2)*np.square(grads['db' + str(l+1)])

            # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
            s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)]/(1 - beta2**t)
            s_corrected["db" + str(l+1)] = s["db" + str(l+1)]/(1 - beta2**t)

            # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*v_corrected["dW" + str(l+1)]/(np.sqrt(s_corrected["dW" + str(l+1)])+epsilon)
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*v_corrected["db" + str(l+1)]/(np.sqrt(s_corrected["db" + str(l+1)])+epsilon)

            return parameters, v, s
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

def nn_model(X, Y, n_h, learning_rate, optimizer='adam', epochs = 10000, print_cost=False):
    """
    Arguments:
    X; dataset of shape (number of examples, n_features)
    Y; labels of shape (number of examples, 1)
    n_h; size of the hidden layer
    num_iterations; Number of iterations in gradient descent loop
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict."""
    
    
    n_x, n_y = layer_size(X, Y, n_h)[0], layer_size(X, Y, n_h)[2]
    parameters = init_parameters(n_x, n_h, n_y)
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    t=0
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    for l in range(L):
    
        v["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0], parameters["W" + str(l+1)].shape[1]))
        v["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0], parameters["b" + str(l+1)].shape[1]))
        s["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0], parameters["W" + str(l+1)].shape[1]))
        s["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0], parameters["b" + str(l+1)].shape[1]))
    
    for i in range(epochs+1):
        
        #Run forward_prop:
        activation2, cache = forward_propagation(X, parameters)
        
        #Find cost for current weights:
        cost = cost_function(Y, activation2)
        
        #Run backprop to estimate gradients:
        gradients = backpropagation(parameters, cache, X, Y)
        
        if optimizer != 'adam':
            #Update current parameters:
            parameters = parameters_update(parameters, gradients, learning_rate, 
                                                 optimizer, beta1, beta2, epsilon, t, L, v, s)
            
        elif optimizer == 'adam':
            t = t + 1 
            parameters, v, s = parameters_update(parameters, gradients, learning_rate, 
                                                 optimizer, beta1, beta2, epsilon, t, L, v, s)
            
            
        if print_cost and i % 10 == 0:
            print("Cost after epoch %i: %f" %(i, cost))
        
    return parameters


def predict(parameters, X):
    """
    Use forward_prop to predict after model tuning.

    """
    
    A2 = forward_propagation(X, parameters)[0]

    predictions = A2
    
    return predictions

*The Boston Housing Dataset*

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns:

CRIM - per capita crime rate by town

ZN - proportion of residential land zoned for lots over 25,000 sq.ft.

INDUS - proportion of non-retail business acres per town.

CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)

NOX - nitric oxides concentration (parts per 10 million)

RM - average number of rooms per dwelling

AGE - proportion of owner-occupied units built prior to 1940

DIS - weighted distances to five Boston employment centres

RAD - index of accessibility to radial highways

TAX - full-value property-tax rate per $10,000

PTRATIO - pupil-teacher ratio by town

B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

LSTAT - % lower status of the population

MEDV - Median value of owner-occupied homes in $1000's

In [3]:
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
housing = pd.read_csv("boston housing.csv", header=None, delim_whitespace=True, names=columns)

In [4]:
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
# Let's scale the columns before plotting them against MEDV
min_max_scaler = preprocessing.MinMaxScaler()
column_sels = ['LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE']
x = housing.loc[:,column_sels]
y = housing['MEDV']

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(housing, y, train_size = 0.6, random_state = 20)
scaler = MinMaxScaler((-1,1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
y_train =y_train.values.reshape(len(y_train.values),1)
y_test = y_test.values.reshape(len(y_test.values),1)

In [9]:
model = nn_model(x_train, y_train, 200, 0.001, optimizer='none', epochs = 200, print_cost=True)

Cost after epoch 0: 606.854970
Cost after epoch 10: 83.397058
Cost after epoch 20: 83.046159
Cost after epoch 30: 82.996189
Cost after epoch 40: 82.942895
Cost after epoch 50: 82.884869
Cost after epoch 60: 82.820762
Cost after epoch 70: 82.749097
Cost after epoch 80: 82.668221
Cost after epoch 90: 82.576255
Cost after epoch 100: 82.471125
Cost after epoch 110: 82.350533
Cost after epoch 120: 82.212067
Cost after epoch 130: 82.055399
Cost after epoch 140: 81.885626
Cost after epoch 150: 81.706236
Cost after epoch 160: 81.517228
Cost after epoch 170: 81.315527
Cost after epoch 180: 81.116824
Cost after epoch 190: 80.927141
Cost after epoch 200: 80.740675


In [10]:
cost = 0
n = 0
for i,j in zip(predict(model, x_test), y_test):
    MSE = (i - j)**2
    cost += np.sum(MSE)
    n += 1 
cost = cost / n
cost

86.84786815974407